## VacationPy

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame

In [ ]:
# Load weather data from 'weatherpy'
file = "Cities.csv"
data = pd.read_csv(file)
data

### Humidity Heatmap

In [ ]:
# Access map with API key
gmaps.configure(api_key=g_key)

# DATA
coordinates = data[['Latitude', 'Longitude']]
humidity = data['Humidity']

# Size of figure
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Heat layer
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False,
                                max_intensity=10, point_radius=1)

# Add the layer to the map
fig.add_layer(heat_layer)
fig

### Create new DataFrame fitting weather criteria

In [ ]:
# This data for vacations... yaaay
weather_conditions = data[(data['Cloudiness'] > 40) & (data['Humidity'] < 40)
                             & (data['Max Temperature'] < 90) & (data['Wind Speed'] < 15)]

# Latitude and longitude data
weather_cond_coord =  weather_conditions[['Latitude', 'Longitude']]
weather_conditions

### Hotel Map

In [ ]:
# Hotels information for the vacation cities
hotel_data = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

for index,row in weather_cond_coord.iterrows():
    #dataset
    x = row[0] #lat
    y = row[1] #lng
    xy = f"{x},{y}"
    #print(xy)
    
    # call API
    params = {"location": xy,
          "radius": "5000",
          "keyword": "hotel",
          "key": g_key
             }
    try:
        response = requests.get(base_url, params=params).json()
        hotel_data.append(response['results'][0]['name'])
    except:
        hotel_data.append(np.nan)

hotel_data

In [ ]:
# Add hotel information to dataframe
hotel_df = pd.Series(hotel_data)
weather_conditions['Hotel Name'] = hotel_df.values # :E

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in weather_conditions.iterrows()]
locations = weather_conditions[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

# Size of figure
figure_layout = {
    'width': '800px',
    'height': '500px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weather_conditions['Humidity'], dissipating=False,
                                max_intensity=10, point_radius=1)

# Add the layer to the map
fig.add_layer(heat_layer)
#Display
fig